In [1]:
import os
import sys
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

%matplotlib inline

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
latiss = LATISS()

In [4]:
await latiss.start_task

Read historical data in 7.32 sec
Read historical data in 7.71 sec
Read historical data in 8.05 sec
Read historical data in 8.59 sec


[None, None, None, None]

In [ ]:
await salobj.set_summary_state(latiss.atarchiver, salobj.State.ENABLED, settingsToApply="normal")

In [5]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing


In [6]:
# This should start at 068 (I think)
# await latiss.take_bias(nbias=10) # Tiago has fixed the 3 s delay, so this should work

for etime in [1.0, 10.0, 100.0]:
    print(f"Taking 5 darks with exptume {etime}")
    await latiss.take_darks(exptime=etime, ndarks=5)

for i in range(25):
    etime = 0.2 * (i + 1)
    print(f"Sequence {i}. Taking 4 flats with exptime {etime}")
    try:
        await latiss.take_flats(exptime=etime, nflats=4)
    except asyncio.TimeoutError:
        print(f"Timeout taking flat with exptime {etime}")
        continue

Taking 5 bias with exptume 1.0
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Taking 5 bias with exptume 10.0
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Taking 5 bias with exptume 100.0
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Sequence 0. Taking 4 flats with exptime 0.2
Waiting for header service LFO before continuing

In [7]:
print("done")

done


In [8]:
latiss.take_flats()

Signature:
latiss.take_flats(
    exptime,
    nflats,
    filter=None,
    grating=None,
    linear_stage=None,
    checkpoint=None,
)
Docstring:
Take a series of flat field images.

Parameters
----------
exptime : `float`
    Exposure time for flats.
nflats : `int`
    Number of flat frames to take.
filter : `None` or `int` or `str`
    Filter id or name. If None, do not change the filter.
grating : `None` or `int` or `str`
    Grating id or name.  If None, do not change the grating.
linear_stage : `None` or `float`
    Linear stage position.  If None, do not change the linear stage.
checkpoint : `coro`
    A optional awaitable callback that accepts one string argument
    that is called before each bias is taken.
File:      ~/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/latiss.py
Type:      method


In [11]:
for i in range(20):
    etime = 0.4 * (i + 1)
    print(f"Sequence {i}. Taking 4 flats with exptime {etime}")
    try:
        await latiss.take_flats(exptime=etime, nflats=4, filter='KPNO_373A_677nm')
    except asyncio.TimeoutError:
        print(f"Timeout taking flat with exptime {etime}")
        continue

Sequence 0. Taking 4 flats with exptime 0.4
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Sequence 1. Taking 4 flats with exptime 0.8
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Sequence 2. Taking 4 flats with exptime 1.2000000000000002
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Sequence 3. Taking 4 flats with exptime 1.6
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) fa

In [10]:
 0.4 * (19 + 1)

8.0

In [ ]:
await latiss.take_flats(exptime=etime, nflats=4, filter='KPNO_373A_677nm')

In [16]:
latiss.take_flats?

Signature:
latiss.take_flats(
    exptime,
    nflats,
    filter=None,
    grating=None,
    linear_stage=None,
    checkpoint=None,
)
Docstring:
Take a series of flat field images.

Parameters
----------
exptime : `float`
    Exposure time for flats.
nflats : `int`
    Number of flat frames to take.
filter : `None` or `int` or `str`
    Filter id or name. If None, do not change the filter.
grating : `None` or `int` or `str`
    Grating id or name.  If None, do not change the grating.
linear_stage : `None` or `float`
    Linear stage position.  If None, do not change the linear stage.
checkpoint : `coro`
    A optional awaitable callback that accepts one string argument
    that is called before each bias is taken.
File:      ~/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/latiss.py
Type:      method


In [17]:
latiss.take_image?

Signature:
latiss.take_image(
    exptime,
    shutter,
    image_type,
    group_id,
    filter=None,
    grating=None,
    linear_stage=None,
    science=True,
    guide=False,
    wfs=False,
)
Docstring:
Set up the spectrograph and take a series of images.


Setting up the spectrograph and taking images cannot be done
concurrently. One needs first to setup the spectrograph then,
request images.

Parameters
----------
exptime : `float`
    The exposure time for the image, in seconds.
shutter : `bool`
    Should activate the shutter? (False for bias and dark)
image_type : `str`
    Image type (a.k.a. IMGTYPE) (e.g. e.g. BIAS, DARK, FLAT, FE55,
    XTALK, CCOB, SPOT...)
group_id : `str`
    Image groupId. Used to fill in FITS GROUPID header
grating : `None` or `int` or `str`
    Grating id or name.  If None, do not change the grating.
linear_stage : `None` or `float`
    Linear stage position.  If None, do not change the linear stage.
science : `bool`
    Mark image as science (default

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [15]:
await latiss.take_image(exptime=1., shutter=True,image_type="OBJECT", group_id="OrionNebula")

Waiting for header service LFO before continuing


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


In [18]:
def end_readout_callback(data):
    print(data.imageName, data.exposureTime)

In [19]:
latiss.atcamera.evt_endReadout.callback = end_readout_callback

AT_O_20200127_000500 30.0
AT_O_20200127_000501 3.0
AT_O_20200127_000502 30.0
AT_O_20200127_000503 3.0
AT_O_20200127_000504 30.0
AT_O_20200127_000505 3.0
AT_O_20200127_000506 30.0
AT_O_20200127_000507 3.0
AT_O_20200127_000508 30.0
AT_O_20200127_000509 120.0
AT_O_20200127_000510 30.0
AT_O_20200127_000511 30.0
AT_O_20200127_000512 30.0
AT_O_20200127_000513 30.0
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
AT_O_20200127_000514 30.0
AT_O_20200127_000515 30.0
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
AT_O_20200127_000516 30.0
AT_O_20200127_000517 30.0
AT_O_20200127_000518 30.0
AT_O_20200127_000519 20.0
AT_O_20200127_000520 2.0
AT_O_20200127_000521 2.0
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
AT_O_20200127_000522 20.0
AT_O_20200127_000523 20.0
AT_O_20200127_000524 20.0
AT_O_20200127_000525 20.0
AT_O_20200127_000526 20.0


In [21]:
await salobj.set_summary_state(latiss.atspectrograph, salobj.State.STANDBY, settingsToApply="current")

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [22]:
await salobj.set_summary_state(latiss.atspectrograph, salobj.State.ENABLED, settingsToApply="current")

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

AT_O_20200127_000527 20.0
AT_O_20200127_000528 20.0
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
AT_O_20200127_000529 20.0
AT_O_20200127_000530 2.0
AT_O_20200127_000531 2.0
AT_O_20200127_000532 2.0
AT_O_20200127_000533 2.0
AT_O_20200127_000534 2.0
AT_O_20200127_000535 2.0
AT_O_20200127_000536 20.0
AT_O_20200127_000537 20.0
AT_O_20200127_000538 20.0
AT_O_20200127_000539 2.0
AT_O_20200127_000540 2.0
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
AT_O_20200127_000541 2.0
AT_O_20200127_000542 20.0
AT_O_20200127_000543 20.0
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
AT_O_20200127_000544 20.0
AT_O_20200127_000545 0.0
AT_O_20200127_000546 0.0
AT_O_20200127_000547 0.0
AT_O_20200127_000548 0.0
AT_O_20200127_000549 0.0
AT_O_20200127_000550 0.0
AT_O_20200127_000551 0.0
AT_O_20200127_000552 0.0
AT_O_20200127_000553 0.0
AT_O_20200127_000554 0.0
AT_O_20200127_000555 2.0
RemoteEvent(ATHeaderService, 0, log